In [1]:
# import models from the Preetum Nakkiran paper 

from models.mcnn import make_cnn
from models.resnet18k import make_resnet18k

In [2]:
import copy

In [8]:
import torch.nn as nn

In [3]:
import torch

In [4]:
import numpy as np

In [3]:
dummy_cnn=make_cnn()

In [24]:
p=next(dummy_cnn[0].parameters())

In [31]:
p[0].dtype

torch.float32

In [32]:
import itertools

In [34]:
type(itertools.product([1,2],[3,4]))

itertools.product

In [49]:
from torchvision.datasets import CIFAR10

In [50]:
c10_tr=CIFAR10('data/',download=True,
              transform=lambda x: copy.copy(np.asarray(x,dtype=np.float32)/255).transpose(2,0,1))

Files already downloaded and verified


In [42]:
c10_tr[0][0].shape

(3, 32, 32)

In [14]:
mi=torch.rand((1,2,32,32))

In [31]:
miy=torch.rand((1))

In [32]:
miy[0]=9

In [43]:
miy=miy.to(torch.int64)

In [44]:
miy

tensor([9])

In [50]:
_cnn=make_cnn(10,10,2)

In [54]:
_cnn(mi).shape==(1,10)

True

In [5]:
import lightning.pytorch as pl

In [11]:
class LitModel(pl.LightningModule):
    def __init__(self,model_class,lr,loss,c,n_class=10,ch=3):
        super().__init__()
        self.model=model_class(c,n_class,ch)
        self.loss=loss
        self.lr=lr
        self.save_hyperparameters(ignore=['loss'])
        
    def forward(self,xbatch):
        return self.model(xbatch)
    
    def pretty_loss(self,yhat,y):
        return self.loss(yhat,y).sum()/y.shape[0]
    
    def training_step(self,batch):
        x,y=batch
        y_hat=self.forward(x)
        loss=self.pretty_loss(y_hat,y)
        
        self.log("train loss",loss)
        return loss
    
    def validation_step(self,batch):
        x,y=batch
        y_hat=self.model(x)
        loss=self.pretty_loss(y_hat,y)
        
        self.log("valid loss",loss)
        
    def configure_optimizers(self):
        optimizer=torch.optim.Adam(self.model.parameters(),lr=self.lr)
        return optimizer

In [16]:
l_cnn=LitModel(make_cnn,1e-4,nn.CrossEntropyLoss(reduction='none'),10,ch=2)

In [28]:
y_hat=l_cnn(mi)

In [40]:
torch.Tensor([1]).dtype

torch.float32

In [46]:
l_cnn.pretty_loss(y_hat,miy)

tensor(5.4850, grad_fn=<DivBackward0>)

In [30]:
help(nn.CrossEntropyLoss)

Help on class CrossEntropyLoss in module torch.nn.modules.loss:

class CrossEntropyLoss(_WeightedLoss)
 |  CrossEntropyLoss(weight: Union[torch.Tensor, NoneType] = None, size_average=None, ignore_index: int = -100, reduce=None, reduction: str = 'mean', label_smoothing: float = 0.0) -> None
 |  
 |  This criterion computes the cross entropy loss between input logits
 |  and target.
 |  
 |  It is useful when training a classification problem with `C` classes.
 |  If provided, the optional argument :attr:`weight` should be a 1D `Tensor`
 |  assigning weight to each of the classes.
 |  This is particularly useful when you have an unbalanced training set.
 |  
 |  The `input` is expected to contain the unnormalized logits for each class (which do `not` need
 |  to be positive or sum to 1, in general).
 |  `input` has to be a Tensor of size :math:`(C)` for unbatched input,
 |  :math:`(minibatch, C)` or :math:`(minibatch, C, d_1, d_2, ..., d_K)` with :math:`K \geq 1` for the
 |  `K`-dimensio

In [47]:
from torch.utils.data import DataLoader

In [52]:
x,y=next(iter(DataLoader(c10_tr,batch_size=5)))

In [54]:
x.shape

torch.Size([5, 3, 32, 32])

In [55]:
y.shape

torch.Size([5])

In [56]:
y

tensor([6, 9, 9, 4, 1])

In [58]:
x.dtype

torch.float32

In [59]:
y.dtype

torch.int64

In [63]:
def supposed_loss(y_hat,y):
    _exp=torch.exp(y_hat)
    return -torch.log(_exp[y]/_exp.sum())

In [79]:
loss_equal(supposed_loss(y_hat[0],miy),2*l_cnn.pretty_loss(y_hat,miy))

False

In [64]:
supposed_loss(y_hat[0],miy)

tensor([5.4850], grad_fn=<NegBackward0>)

In [66]:
l_cnn.pretty_loss(y_hat,miy)==supposed_loss(y_hat[0],miy)

tensor([False])

In [67]:
def generate_mock_batch(n_batch,n_class=10,ch=3):
    # generate a mock batch of n (32,32) images of ch channels

    x=torch.rand(n_batch,ch,32,32)
    y=torch.randint(0,n_class,(n_batch,))

    return (x,y)

(tensor([[[[0.4223, 0.2090, 0.5440,  ..., 0.6503, 0.7455, 0.6144],
           [0.9687, 0.3051, 0.5587,  ..., 0.4188, 0.8277, 0.0753],
           [0.8893, 0.9324, 0.7278,  ..., 0.7050, 0.2801, 0.9089],
           ...,
           [0.4760, 0.1242, 0.6057,  ..., 0.0189, 0.6819, 0.2683],
           [0.1343, 0.6134, 0.7260,  ..., 0.7422, 0.5964, 0.2288],
           [0.1627, 0.7954, 0.0148,  ..., 0.1963, 0.2646, 0.1940]],
 
          [[0.9685, 0.1075, 0.7943,  ..., 0.7306, 0.1830, 0.4847],
           [0.5192, 0.6793, 0.7004,  ..., 0.2126, 0.2751, 0.5166],
           [0.9722, 0.8445, 0.6431,  ..., 0.3674, 0.2850, 0.8842],
           ...,
           [0.9647, 0.7551, 0.0253,  ..., 0.9557, 0.6224, 0.4244],
           [0.7053, 0.3447, 0.5657,  ..., 0.7056, 0.2891, 0.1458],
           [0.4321, 0.5315, 0.5292,  ..., 0.1724, 0.1113, 0.8790]],
 
          [[0.6246, 0.5046, 0.8729,  ..., 0.2356, 0.9551, 0.7282],
           [0.4095, 0.6777, 0.6463,  ..., 0.9402, 0.3531, 0.3632],
           [0.5684, 0.50

In [77]:
def loss_equal(l1,l2):
    # sets equality criterion for floating loss tensors

    diff=torch.abs(l1-l2)
    sum_abs=torch.abs(l1)+torch.abs(l2)

    try:
        return bool((diff/sum_abs<1e-4))
    except:
        return True

In [82]:
def test_trainstep(model_fun,n_class,n_batch):
    model=LitModel(model_fun,1e-4,nn.CrossEntropyLoss(reduction='none'),10,n_class,3)
    x,y=generate_mock_batch(n_batch,n_class,3)
    y_hat=model(x)

    sum_loss=0
    for i in range(n_batch):
        print(supposed_loss(y_hat[i],y[i]))
        sum_loss+=supposed_loss(y_hat[i],y[i])
    print('-------',sum_loss/10)
    print(model.training_step((x,y)))
    
    

In [83]:
test_trainstep(make_cnn,10,10)

tensor(4.8363, grad_fn=<NegBackward0>)
tensor(2.3048, grad_fn=<NegBackward0>)
tensor(1.5867, grad_fn=<NegBackward0>)
tensor(3.0052, grad_fn=<NegBackward0>)
tensor(2.3255, grad_fn=<NegBackward0>)
tensor(3.0569, grad_fn=<NegBackward0>)
tensor(3.3199, grad_fn=<NegBackward0>)
tensor(2.5288, grad_fn=<NegBackward0>)
tensor(0.9629, grad_fn=<NegBackward0>)
tensor(0.8305, grad_fn=<NegBackward0>)
------- tensor(2.4757, grad_fn=<DivBackward0>)
tensor(2.4757, grad_fn=<DivBackward0>)
